In [1]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import rnn

(corpus_indices, char_to_idx, idx_to_char,
 vocab_size) = d2l.load_data_jay_lyrics()

In [28]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                nd.zeros(num_hiddens, ctx=ctx))

    W_xz, W_hz, b_z = _three()  # 更新门参数
    W_xr, W_hr, b_r = _three()  # 重置门参数
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

In [3]:
# 返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [4]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + nd.dot(R * H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [25]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 16, 64, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [10]:
# 完全实现
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 172.302138, time 0.73 sec
 - 分开 我想不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我
 - 不分开 我想不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我不不 我
epoch 80, perplexity 45.090068, time 0.73 sec
 - 分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 120, perplexity 9.393079, time 0.71 sec
 - 分开我 不要再这样打我妈妈 我的你 你给我抬起睡是你想 不要再这样打我妈妈 我的你 你给我抬起睡有 有你
 - 不分开我 相你 是你开的玩笑 我想揍你已经很久 想想和你已经着我 别发 你想我久难难  没有你烦我有多烦多
epoch 160, perplexity 2.540423, time 0.71 sec
 - 分开我 不要到这队友 唱的让我 你你的可爱在一元 风力在风离写香日记    穿手了里像始一句 漫拉上的字
 - 不分开 为作一直重 我来一碗热活 我爱你这生活 后知悄悄 又过风觉演走 它在情里池走轻轻叹息 太多的我爱你


In [26]:
# 简洁实现
gru_layer = rnn.GRU(num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size)
# 该函数只有相邻取样的
d2l.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 116.304156, time 0.10 sec
 - 分开 我不能 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
 - 不分开 我不能 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 我不 
epoch 80, perplexity 18.370633, time 0.11 sec
 - 分开我妈妈的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏
 - 不分开我想要你 爱情我的太膀 你 在我胸口睡著你 我想想你已经很久 想想想你想你 我不要再想你 我不要再想
epoch 120, perplexity 2.423426, time 0.11 sec
 - 分开 我想就这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带你的肩美美问龙卷卷风 离不开暴风圈
 - 不分开 整颗我一起重 后知后觉 后知后觉 迷慢是停留 我该好好生活 静静悄悄默默离开 陷入了危险边缘 该攻
epoch 160, perplexity 1.161219, time 0.11 sec
 - 分开 它在空中停留 所有人看着我 抛物线进球 单手过人运球 篮下妙传出手 漂亮的假动作 帅呆了我 全场盯
 - 不分开 整已经离开我 不知不觉 我跟了这节奏 后知后觉 后知后觉 迷知后觉 迷不懂 不想就这样牵我 不知不


In [29]:
# 简洁实现
gru_layer = rnn.GRU(num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size)
# 该函数只有相邻取样的
d2l.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 122.545908, time 0.07 sec
 - 分开 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我
 - 不分开 我想你 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我不要 我
epoch 80, perplexity 19.346504, time 0.07 sec
 - 分开 我想要你的微笑 像想想 你爱我 我想要 爱你的让我面红的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
 - 不分开 爱你的让我面狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱
epoch 120, perplexity 3.004208, time 0.07 sec
 - 分开 一直心酒 在人村中 恨人耿空 一切落痛 在人忆中的溪肉 我说店小二 三两银够不够 景色入秋 快使用
 - 不分开 说没有一个人慢慢 我的太笑 你爱我早已经要 我想要你的微笑每天都能看到  我知道这里很美但家乡的你
epoch 160, perplexity 1.255027, time 0.07 sec
 - 分开 我想带你已经车 我想要你的微笑 我想要你的微笑每天都能看到  我知道这里很美但家乡的你更美走来我只
 - 不分开  没有回忆对我的愿望就怎么小 就怎么每天祈祷我的心跳你知道  杵在伊斯坦堡 却只想你和汉堡 我想要
